# Getting Started: Multi-Task Decoding (Task 3)

This notebook takes in an encoder and computes R2 scores between embeddings and different Semantic features as part of of Task 3.

### Install the required packages

In [8]:
#Download and install Pytorch for CUDA 10.2
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu102
#Download and install Pytorch for CUDA 11.3
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113
    
!pip install -e ../

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu102
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining file:///C:/Users/Christopher%20Liding/Documents/GitHub/MTNeuro
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached https://github.com/qubvel/segmentation_models.pytorch/archive/740dab561ccf54a9ae4bb5bda3b8b18df3790025.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Attempting uninstall: mtneuro
    Found existing installation: mtneuro 1.0.0
    Uninstalling mtneuro-1.0.0:
      Successfully uninstalled mtneuro-1.0.0
  Running setup.py develop for mtneuro


### Import the Required Packages

In [9]:
#Import libraries
import os
import sys
import json
import torch
import math
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import argparse
import umap

#PyTorch imports
from torch.utils.data import DataLoader
import torch.multiprocessing as mp
import torch.nn.functional as F

#Sci-kit learn imports
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA

#MTNeuro modules
sys.path.append('../')                 #setting the location to look for the required packages
from MTNeuro.annots.features import extract_cell_stats,extract_axon_stats,extract_blood_stats
from MTNeuro.annots.get_cutouts import get_cutout_data
from MTNeuro.annots.latents import get_latents, get_unsup_latents


### Load the Encoder

The encoder file path **and** the encoder type is required here. 

Any of the models used in Task 1 can be used as an encoder. They can be found here: [[Dropbox](https://www.dropbox.com/home/MTNeuro/Models/Task1)]

The encoder types are `ssl`, `supervised`, `PCA`, and `NMF`.

In [10]:
#In this demo, BYOL is used. 
#TODO: Download the model weights for BYOL from the Dropbox and place the file in the same directory as this notebook.

encoder_file_path = "ckpt-800.pt"
encoder_type = "ssl"

#--Do not motify below--
if encoder_type == 'ssl':
    ssl_encoder = 1
    unsupervised = 0
elif encoder_type == 'supervised':
    ssl_encoder =  0
    unsupervised = 0
elif encoder_type == 'PCA': 
    unsupervised = 1
    set_pca = 1
elif encoder_type == 'NMF':
    unsupervised = 1
    set_pca = 0
else:
    print("Incorrectly specified encoder type")

### Specify Cutout Coordinates

Cutout coordinates are specified in the task 3 JSON file found at MTNeuro/taskconfig/task3.json.

In [11]:
config_file_path = "../MTNeuro/taskconfig/task3.json"

try:
    jsonFile = open(config_file_path, 'r')
    slices = json.load(jsonFile)
except IOError:
    print("JSON file not found.")
jsonFile.close()    

#### Downloading Annotations Specified in JSON File

Each cutout from each region of the brain has shape (Z,Y,X) = (360,256,256). 

All 4 cutouts are concatenated together along the z-axis to form `data_array_raw` and `data_array_anno`, which both have shape (1440,256,256).

In [12]:
#Retrieving Data from JSON
xrange_list = [slices['xrange_cor'],slices['xrange_stri'],slices['xrange_vp'],slices['xrange_zi']]
yrange_list = [slices['yrange_cor'],slices['yrange_stri'],slices['yrange_vp'],slices['yrange_zi']]
class_list = ["Cortex","Striatum","VP","ZI"]
zrange = slices['zrange']


boss_dict = {}
boss_dict['image_chan']=slices['image_chan']
boss_dict['annotation_chan'] = slices['annotation_chan']

data_array_raw = []
data_array_anno = []
label_array  = []
up_sample = 4 

#Pulling Data from BossDB
for i in range(0,len(xrange_list)):
    cutout_data_raw,cutout_data_anno = get_cutout_data(xrange_list[i],yrange_list[i],zrange,name=class_list[i])
    
    data_raw = cutout_data_raw[:,:,:]
    data_anno = cutout_data_anno[:,:,:]
    data_array_raw = np.concatenate((data_array_raw,data_raw),axis =0 ) if len(data_array_raw) else data_raw 
    
    data_array_anno = np.concatenate((data_array_anno,data_anno),axis =0 ) if len(data_array_anno) else data_anno
    
    labels = i*np.ones(up_sample*len(data_raw)).reshape(-1,)
    label_array  = np.concatenate((label_array ,labels),axis =0) if len(labels) else labels_train

c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\remote\remote.py:100: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  cfg_parser.readfp(config_handle)


c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\remote\remote.py:100: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  cfg_parser.readfp(config_handle)


c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\remote\remote.py:100: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  cfg_parser.readfp(config_handle)


c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\remote\remote.py:100: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  cfg_parser.readfp(config_handle)


c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)
c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\intern\service\boss\v1\volume.py:299: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data_mat = np.fromstring(raw_data, dtype=resource.datatype)


### Extracting Features and Calculating Linear Readout Scores

In [13]:
print('Extracting cell stats...')
stats_cell= extract_cell_stats(np.copy(data_array_anno))

print('Extracting axon stats...')
stats_axon = extract_axon_stats(np.copy(data_array_anno))

print('Extracting blood stats...')
stats_blood = extract_blood_stats(np.copy(data_array_anno))

'get results for different encoders'
if encoder_type == 'ssl' or encoder_type == 'supervised':
    embeddings = get_latents(data_array_raw,encoder_file_path,ssl_encoder)
elif encoder_type == 'PCA' or encoder_type == 'NMF':
    embeddings = get_unsup_latents(data_array_raw,set_pca)

'Get linear readout scores'
X = embeddings

y = stats_blood[:,1]
reg = LinearRegression().fit(X,y)
blood_vsl_score = reg.score(embeddings,stats_blood[:,1])
print("Blood Vessel Score : {}".format(blood_vsl_score ))

y = stats_cell[:,1]
reg = LinearRegression().fit(X,y)
numb_cell = reg.score(embeddings,stats_cell[:,1])
print("Cell count:{}".format(numb_cell))

y = stats_cell[:,2]
reg = LinearRegression().fit(X,y)
avg_dist_nn_cell = reg.score(embeddings,stats_cell[:,2])
print("cell stat 2 (Avg distance to NN) :{}".format(avg_dist_nn_cell ))

y = stats_cell[:,4]
reg = LinearRegression().fit(X,y)
cell_size = reg.score(embeddings,stats_cell[:,4])
print("Cell Size :{}".format(cell_size))

y = stats_axon[:,1]
reg = LinearRegression().fit(X,y)
axon_rslt = reg.score(embeddings,stats_axon[:,1])
print("Axon  : {}".format(axon_rslt ))

Extracting cell stats...
Extracting axon stats...
Extracting blood stats...


c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ C:\Users\Christopher Liding\AppData\Local\Temp\ipykernel_27064\4076191281.py:12 in <cell  │
│ line: 11>                                                                                 │
│                                                                                           │
│ [Errno 2] No such file or directory: 'C:\\Users\\Christopher                              │
│ Liding\\AppData\\Local\\Temp\\ipykernel_27064\\4076191281.py'                             │
│                                                                                           │
│ c:\users\christopher liding\documents\github\mtneuro\MTNeuro\annots\latents.py:29 in      │
│ get_latents                                                                               │
│                                                                                           │
│   26 │   │   │   │   │   │   │   │    drop_last=False, pin_memory=True)                   │
│   27 │                                                                                    │
│   28 │                                                                                    │
│ ❱ 29 │   encoder = resnet_xray('resnet18').to(torch.device('cuda:0'))                     │
│   30 │   if ssl == 1:                                                                     │
│   31 │   │   ckpt_epoch = BYOLTrainer.load_trained_encoder(encoder, ckpt_path= encoder_fi │
│   32 │   encoder.eval()                                                                   │
│                                                                                           │
│ c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\tor │
│ ch\nn\modules\module.py:907 in to                                                         │
│                                                                                           │
│    904 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)         │
│    905 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() e │
│    906 │   │                                                                              │
│ ❱  907 │   │   return self._apply(convert)                                                │
│    908 │                                                                                  │
│    909 │   def register_backward_hook(                                                    │
│    910 │   │   self, hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]    │
│                                                                                           │
│ c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\tor │
│ ch\nn\modules\module.py:578 in _apply                                                     │
│                                                                                           │
│    575 │                                                                                  │
│    576 │   def _apply(self, fn):                                                          │
│    577 │   │   for module in self.children():                                             │
│ ❱  578 │   │   │   module._apply(fn)                                                      │
│    579 │   │                                                                              │
│    580 │   │   def compute_should_use_set_data(tensor, tensor_applied):                   │
│    581 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):    │
│                                                                                           │
│ c:\Users\Christopher Liding\AppData\Local\Programs\Python\Python310\lib\site-packages\tor │
│ ch\nn\modules\module.py:578 in _apply                                                     │
│                                                   